In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests

def single(tot):
    ##number that determines where the start is, or in other words what page of the table the url is 
    ##as it increments by 25, page 1 => start=0 page 2 => start=25 page 3 => start=50 etc
    num = 0
    while tot != 0:
        if num == 0:
            url = f'https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2013-10-29&EndDate=2024-10-22&InjuriesChkBx=yes&Submit=Search&start={num}'
            response = requests.get(url)
            html = response.content
            soup = BeautifulSoup(html, 'lxml')
            table = soup.find('table', attrs={'class': 'datatable center'})
            df = pd.read_html(str(table))[0]
            num = num + 25           ##increment to 25 to move onto page 2
            tot = tot - 1            ##reduce tot to see how many pages are left to visit
            time.sleep(2.38)         ##rate limit
        else:
            url = f'https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2013-10-29&EndDate=2024-10-22&InjuriesChkBx=yes&Submit=Search&start={num}'
            response = requests.get(url)
            html = response.content
            soup = BeautifulSoup(html, 'lxml')
            table = soup.find('table', attrs={'class': 'datatable center'})
            df = pd.concat([df, pd.read_html(str(table))[0]], ignore_index=True)
            num = num + 25           ##increment the start to go to the next page
            tot = tot - 1            ##reduce the total to determine number of remaining pages
            time.sleep(2.38)         ##rate limit 

    df.columns = ['Date','Team','Acquired','Relinquished','Notes']
    df = df[df['Date']!='Date']
    df.reset_index(drop=True, inplace=True)
    df.Acquired = df.Acquired.str.replace('•','') # remove • in front of player's name
    df.Relinquished = df.Relinquished.str.replace('•','') # remove • in front of player's name
    df['Date'] = pd.to_datetime(df['Date'].str.strip(), format='%Y-%m-%d')
    return df

In [ ]:
df = single(1507)

In [9]:
df.to_csv('2013-2024_injury_stats.csv', index = True)

In [13]:
df = pd.read_csv("2013-2024_injury_stats.csv")

In [2]:
import sys
sys.path.append('./basketball_reference_scraper/basketball_reference_scraper')

In [3]:
import pandas as pd
from bs4 import BeautifulSoup, Comment

try:
    from constants import TEAM_TO_TEAM_ABBR, TEAM_SETS
    from utils import remove_accents
    from request_utils import get_wrapper
except:
    from basketball_reference_scraper.constants import TEAM_TO_TEAM_ABBR, TEAM_SETS
    from basketball_reference_scraper.utils import remove_accents
    from basketball_reference_scraper.request_utils import get_wrapper

import time

def extract_table_from_page(url, table_id, max_retries=10, delay=2):
    for attempt in range(max_retries):
        r = get_wrapper(url)
        if r.status_code == 200:
            break
        else:
            print(f"Attempt {attempt + 1} failed with status {r.status_code}")
            time.sleep(delay)
    else:
        # If the loop didn't break, all attempts failed
        raise ConnectionError('Request to basketball reference failed after retries.')

    soup = BeautifulSoup(r.content, 'lxml')

    # First try direct HTML
    table = soup.find('table', {'id': table_id})
    if table:
        return pd.read_html(str(table))[0]

    # If not found, look inside HTML comments
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    for comment in comments:
        if table_id in comment:
            comment_soup = BeautifulSoup(comment, 'lxml')
            table = comment_soup.find('table', {'id': table_id})
            if table:
                return pd.read_html(str(table))[0]

    raise ValueError(f'Table with id="{table_id}" not found on page (or in HTML comments).')

In [9]:
def extract_injury_data(season):
    main_url = f'https://www.spotrac.com/nba/injured/_/year/{season}/view/player'
    rest_url = f'https://www.spotrac.com/nba/injured/_/year/{season}/view/rest'
    main_table = extract_table_from_page(main_url, 'table')
    rest_table = extract_table_from_page(rest_url, 'table')
    main_table = main_table.drop(columns=['Rank', 'Cash Total Per Days Missed'])
    rest_table = rest_table.drop(columns=['Rank', 'Reasons', 'Cash Total Per Days Missed'])
    return main_table, rest_table

In [ ]:
seasons = {i : [] for i in range(2016, 2024)}
for i in range(2016, 2024):
    main, rest = extract_injury_data(i)
    seasons[i] = [main, rest]

In [14]:
import pickle

def save_pickle(structure, name):
    # Save
    with open(f'{name}.pkl', 'wb') as f:
        pickle.dump(structure, f)

def load_pickle(file):
    with open(file, 'rb') as f:
        loaded_data = pickle.load(f)
        return loaded_data

In [ ]:
save_pickle(seasons, 'seasons')

In [16]:
seasons

{2016: [              Player Pos Team      Reason  Games Missed  \
  0         Chris Bosh  PF  MIA     Illness            82   
  1     Nikola Pekovic   C  MIN    Achilles            82   
  2        Ian Mahinmi   C  WAS  Knee, Calf            51   
  3    Khris Middleton  SF  MIL   Hamstring            50   
  4     Jerryd Bayless  PG  PHI       Wrist            79   
  ..               ...  ..  ...         ...           ...   
  330     Yogi Ferrell  PG  DAL       Ankle             1   
  331   Semaj Christon  PG  OKC  Concussion             1   
  332     A.J. Hammons   C  DAL     Illness             1   
  333   Chasson Randle  PG  NYK       Ankle             1   
  334    Daniel Ochefu  SF  WAS     Illness             1   
  
                                          Days Missed  
  0                                 169 10/26/16-4/12  
  1                                 169 10/26/16-4/12  
  2    103 10/27/16-11/25, 11/28/16-2/7, 4/12/17-4/12  
  3                                

In [14]:
get_game_logs('Chris Paul', 2018)

You searched for "Chris Paul"
15 results found.
/players/p/paulch01
https://www.basketball-reference.com//players/p/paulch01/gamelog/2018
89


C:\Users\tongf\AppData\Local\Temp\ipykernel_29252\3633985672.py:71: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


,Gcar,DATE,Team,HOME/AWAY,OPPONENT,Result,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GAME_SCORE,+/-
0,835,2017-10-17,HOU,AWAY,GSW,"W, 122-121",*,33:04,2,9,...,7,8,10,2,1,1,4,4,8.4,-13
1,NaN,2017-10-18,HOU,AWAY,SAC,"W, 105-100",Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play
2,NaN,2017-10-21,HOU,HOME,DAL,"W, 107-91",Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
3,NaN,2017-10-23,HOU,HOME,MEM,"L, 90-98",Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
4,NaN,2017-10-25,HOU,AWAY,PHI,"W, 105-104",Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,890,2018-04-05,HOU,HOME,POR,"W, 96-94",*,31:07,11,19,...,4,4,5,0,0,2,2,27,20.0,1
79,891,2018-04-07,HOU,HOME,OKC,"L, 102-108",*,35:29,6,15,...,3,4,9,1,0,3,4,17,13.2,3
80,892,2018-04-10,HOU,AWAY,LAL,"W, 105-99",*,27:14,9,14,...,3,3,6,1,0,0,3,22,20.7,2
81,NaN,2018-04-11,HOU,AWAY,SAC,"L, 83-96",Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play
